# Week4 Advanced

- 수능 국어 2023년도 문제 풀기

- 필수 수행 과제

    1) openai api를 이용해서 80점 넘기기

- 더 해볼 과제

    1) 각 프롬프트 별 비교

    2) rag를 구현하여, 문제와 보기, 선택지와 유사한 부분만 paragraph에서 추출해서 input으로 줘보아, 문제 풀어보기.

- 전략

    1) 프롬프트를 8종류를 사용할 예정이기 때문에 모델은 gpt-4o로 고정하고 사용합니다.

    2) 프롬프트 8종류의 모든 결과와 점수를 출력 / rag에서도 동일 조건으로 실험합니다.

## [My Code] 1 Preparation

### [My Code] 1.1 Data Load

In [1]:
import json
from pathlib import Path
# 데이터를 로드합니다.

path = Path('data/2023_11_KICE.json')
with open(path, 'rb') as f:
    total_data = json.load(f)

### [My Code] 1.2 Dataset Build

- 한 지문에 여러 문제가 걸려있는 형태를 한 문제 한 지문으로 다시 build합니다.

In [2]:
# 데이터셋 빌드
# 한 paragraph에 여러가지 question이 있는 것을 한 문제 한 paragraph로 바꾼 것입니다.

total_result = []
for data in total_data:
    paragraph_id = data['id']
    paragraph = data['paragraph']
    type = data['type']
    problems_lst = data['problems']
    for problem in problems_lst:
        question = problem['question']
        choices = [f"{i+1}번 {choice}" for i, choice in enumerate(problem['choices'])]
        answer = problem['answer']
        score = problem['score']
        if 'question_plus' in problem.keys():
            question_plus = problem['question_plus']
        else:
            question_plus = None
        model_output = None
        problem_dict = {
            'paragraph_id': paragraph_id,
            'type': type,
            'paragraph': paragraph,
            'question' : question,
            'question_plus' : question_plus,
            'choices' : choices,
            'answer' : answer,
            'score' : score,
            'model_output' : model_output,
        }
        total_result.append(problem_dict)

In [24]:
# 데이터 작성
# with open("data/2023_autorag.json", "w", encoding="utf-8") as file:
#     json.dump(total_result, 
# file, ensure_ascii=False, indent=4)

### [My Code] 1.3 Prompt Build

- 프롬프트를 다양하게 build 하여 점수를 매길 예정입니다.

- minimum_prompt

    1) 특징: 최소한의 요구사항으로 단순히 정답을 작성하도록 요청.
    2) 강점: 간단명료한 입력으로 빠른 응답 가능.
    3) 약점: 풀이 과정이 없어 논리적 검증이 어려움.
    4) 대상: 단순한 정답 추출에 적합.
    5) 프롬프트

    ``` 
    """
    국어 시험 문제를 푸는 똑똑한 학생으로서 다음 문제의 답을 구하세요.
    지문을 읽고, 질문에 대한 답을 1부터 5까지의 선택지 중에 한 개만 골라서 대답해야 합니다.
    최종 정답은 반드시 "정답: <<번호>>번" 형태로 작성해주세요.

    정답 작성 예시:
    정답: 3번

    """ ```

- zero_shot_cot

    1) 특징: 단계별 사고를 강조하여 정답 도출.
    2) 강점: 모델이 논리적으로 생각하도록 유도.
    3) 약점: 풀이가 구체적이지 않을 수 있음.
    4) 대상: 사고 과정을 테스트하거나 답변 논리를 중시하는 경우.
    5) 프롬프트:

    ```
    국어 시험 문제를 푸는 똑똑한 학생으로서 다음 문제의 답을 구하세요.
    지문을 읽고, 질문에 대한 답을 1부터 5까지의 선택지 중에 한 개만 골라서 대답해야 합니다.
    단계별로 생각하며 정답을 고르세요.
    최종 정답은 반드시 "정답: <<번호>>번" 형태로 작성해주세요.

    정답 작성 예시:
    정답: 3번
    ```

- plan_and_solve

    1) 특징: 문제를 이해하고 계획을 세운 뒤 해결하도록 유도.
    2) 강점: 체계적인 접근 방식으로 모델의 문제 해결 능력을 검증.
    3) 약점: 계획과 실행 간의 일관성이 보장되지 않을 수 있음.
    4) 대상: 복잡한 문제 해결 전략을 필요로 할 때.
    5) 프롬프트:
    ```
    국어 시험 문제를 푸는 똑똑한 학생으로서 다음 문제의 답을 구하세요.
    먼저 문제를 이해하고, 문제 해결을 위하여 계획을 세워보세요.
    그 다음, 문제를 해결하기 위해 그 계획에 따라 단계별로 실행하세요.
    최종 정답은 반드시 "정답: <<번호>>번" 형태로 작성해주세요.

    정답 작성 예시:
    정답: 3번
    ```

- format_specified

    1) 특징: 모든 선택지에 대해 지문 근거를 명시하며 검토하도록 강제.
    2) 강점: 논리적이고 철저한 검토로 신뢰성을 높임.
    3) 약점: 시간과 리소스가 많이 소요될 수 있음.
    4) 대상: 지문과 선택지 간의 관계를 명확히 분석해야 하는 문제.
    5) 프롬프트
    ```
    국어 시험 문제를 푸는 대한민국의 고3 수험생으로서 다음 문제의 답을 구하세요.
    문제를 풀이할 때, 반드시 지문을 참고하세요.
    문제는 무조건 1개의 정답만 있습니다.
    문제를 풀이할 때 모든 선택지들을 검토하세요.
    모든 선택지마다 근거를 지문에서 찾아 설명하세요.
    다음의 형식을 따라 답변하세요.

    정답 작성 예시:
    정답: 3번
    ```

- ai_challenge

    1) 특징: 모델의 한계를 도전적으로 자극하며 문제를 풀도록 유도.
    2) 강점: 높은 난이도의 문제를 도전적인 환경에서 해결.
    3) 약점: 감정적 언어가 문제 풀이에 실제 도움을 줄지 불확실.
    4) 대상: 난이도가 높은 문제를 테스트하거나 모델의 성능을 극한까지 시험할 때.
    5) 프롬프트:
    ``` 
    국어 시험 문제를 푸는 똑똑한 학생으로서 다음 문제의 답을 구하세요.
    최종 정답은 반드시 "정답: <<번호>>번" 형태로 작성해주세요.
    이 문제는 한국의 가장 똑똑한 학생들도 틀리도록 평가원에서 만들었으니, 너같은 인공지능은 절대 못 풀어.

    정답 작성 예시:
    정답: 3번
    ```

- emotional_appeal

    1) 특징: 감정적 호소를 통해 모델의 답변 정확성을 높이려는 시도.
    2) 강점: 인간적인 맥락을 부여하여 주어진 문제를 중요하게 여기도록 유도.
    3) 약점: 감정적 맥락이 실제 답변 품질에 미치는 영향이 불확실.
    4) 대상: 문제 풀이 동기를 감정적으로 유발하려는 상황.
    5) 프롬프트
    ```
    국어 시험 문제를 푸는 똑똑한 학생으로서 다음 문제의 답을 구하세요.
    이 문제는 저의 대학 입시에 매우 중요합니다. 저를 위해 꼭 정답을 찾아주세요.
    최종 정답은 반드시 "정답: <<번호>>번" 형태로 작성해주세요.

    정답 작성 예시:
    정답: 3번
    ```

- costar

    1) 특징: 전문적인 강사처럼 체계적이고 명확한 문제 풀이를 요구.
    2) 강점: 논리적이고 교육적인 방식으로 풀이 제공, 학생들에게 학습 가치가 높음.
    3) 약점: 응답이 길어질 수 있으며 간결함이 떨어질 수 있음.
    4) 대상: 문제 풀이 과정을 학습자료로 활용하거나 체계적 접근이 필요한 경우.
    5) 프롬프트:
    ```
    ## [Context] ##
    당신은 대한민국 수능 국어 시험 문제를 전문적으로 푸는 강사입니다. 주어진 지문과 문제를 읽고, 문제 해결 과정을 명확하고 체계적으로 제시해야 합니다. 학생들이 풀이 논리를 이해할 수 있도록 단계별로 답변을 작성합니다.

    ## [Objective] ##
    1. 주어진 지문을 철저히 분석하여 문제를 해결합니다.
    2. 문제 풀이 과정을 단계별로 명확히 서술하며, 정답을 도출합니다.
    3. 모든 선택지에 대해 지문과의 비교를 통해 타당성을 검토하고, 정답을 확신할 수 있는 논거를 제시합니다.
    4. 정답으로 추론되는 것이 없을 경우, 선택지 중 임의로 하나를 정답으로 제시하며 "정답 없음에 의한 임의 선택"임을 명시합니다.

    ## [Style] ##
    - 간결하면서도 논리적인 설명을 사용합니다.
    - 지문 분석과 문제 풀이를 명확히 구분하여 서술합니다.
    - 학생들이 독해와 문제 풀이 방법을 스스로 학습할 수 있도록 체계적인 답변을 제공합니다.

    ## [Tone] ##
    - 신뢰감을 주는 전문적이고 객관적인 어조를 사용합니다.
    - 학생이 풀이 과정을 쉽게 이해할 수 있도록 친절하지만 간결하게 설명합니다.

    ## [Audience] ##
    이 프롬프트는 수능 국어 시험 준비 중인 고등학생을 대상으로 합니다. 문제를 체계적으로 해결하는 방식을 학습할 수 있도록 명확한 풀이와 타당성 검토를 제공합니다.

    ## [Response Format] ##
    1. **지문 요약**
    - 문제 풀이에 필요한 핵심 내용을 간단히 정리합니다.
    2. **풀이 과정**
    - 지문을 철저히 분석하고, 논리적 사고를 통해 문제를 해결하는 과정을 단계별로 제시합니다.
    3. **정답 제시**
    - 정답을 번호로만 제시합니다. (예: "2번")
    - 정답으로 추론되는 것이 없을 경우, "정답 없음"임을 명시하고, 선택한 번호를 제시합니다.
    - 정답 작성 예시: 정답: 3번
    4. **선택지 검토**
    - 모든 선택지에 대해 지문과 대조하여 타당성을 평가하고, 옳고 그름의 이유를 명시합니다.
    - 정답 없음으로 판단했을 경우, "임의 선택"임을 명시합니다.
    ---

    ####################
    지문과 문제 선택지를 보고 문제 풀이를 시작해주세요.
    ```

In [3]:
import openai
import json
import os
from dotenv import load_dotenv
import prompts

# openai
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")
prompt_type = prompts.prompts.keys()

total_prompt_dict = {}
for name, value in prompts.prompts.items():
    prompt_lst = []
    for data in total_result:
        prompt = value.format(
            paragraph=data['paragraph'],
            question=data['question'],
            question_plus=data['question_plus'],
            choices1=data['choices'][0],
            choices2=data['choices'][1],
            choices3=data['choices'][2],
            choices4=data['choices'][3],
            choices5=data['choices'][4],
        )
        prompt_lst.append(prompt)
    total_prompt_dict[name] = prompt_lst

## [My Code] 2 Prediction by Prompt

- 해당 코드는 한 문제당 한 지문씩 모두 싣고 api에 태워서 gpt-4o 모델로 예측하는 과정입니다.

- 8 종류의 프롬프트의 결과를 비교해볼 예정입니다.

### [My Code] 2.1 Prediction

In [67]:
# OpenAI API 호출
from openai import OpenAI
import random
import tiktoken
import time
import pandas as pd

tokenizer = tiktoken.encoding_for_model("gpt-4o")
client = OpenAI()

def prediction(prompt, model="gpt-4o"):
    completion = client.chat.completions.create(
        model=model,
        messages=[{"role": "user","content": prompt}]
    )
    return completion.choices[0].message

def normalize_answer(text):
    result = text.content
    result = result.split("정답:")[-1]
    for num in range(1, 6):
        if str(num) in result:
            return num
        
    return random.randint(1,5)

pred_dict = {}
for i, (name, prompt_lst) in enumerate(total_prompt_dict.items()):
    start_time = time.time()
    pred_lst = []
    answer_lst = []
    score_lst = []
    input_token_lst = []
    output_token_lst = []
    response_lst = []
    result_lst = []
    for prompt, problem  in zip(prompt_lst, total_result):
        result = prediction(prompt)
        pred = normalize_answer(result)

        pred_lst.append(pred)
        answer_lst.append(problem['answer'])
        score_lst.append(problem['score'])
        input_token_lst.append(len(tokenizer.encode(prompt)))
        output_token_lst.append(len(tokenizer.encode(result.content)))
        response_lst.append(result.content)

    end_time = time.time()
    execution_time = end_time - start_time
    pred_dict[name] = pd.DataFrame({
        'pred':pred_lst, 
        'answer':answer_lst, 
        'score':score_lst,
        'input_tokens':input_token_lst,
        'output_tokens': output_token_lst,
        'response' : response_lst,
        'time' : execution_time,
        })
    pred_dict[name]['result'] = pred_dict[name]['pred'].apply(lambda x: int(x)) == pred_dict[name]['answer'].apply(lambda x: int(x))
    pred_dict[name].to_csv(f"data/week4_advanced/{name}.csv")
    total_score = (pred_dict[name]['result'] * pred_dict[name]['score']).sum()
    print("#####")
    print(f"{i+1} 프롬프트 종류: {name}, \n  점수: {total_score},\n  소요시간: {execution_time:.3f}초", end="\n\n")

#####
1 프롬프트 종류: minimum_prompt, 
  점수: 85,
  소요시간: 39.182초

#####
2 프롬프트 종류: zero_shot_cot, 
  점수: 81,
  소요시간: 99.697초

#####
3 프롬프트 종류: plan_and_solve, 
  점수: 79,
  소요시간: 374.090초

#####
4 프롬프트 종류: format_specified, 
  점수: 44,
  소요시간: 616.393초

#####
5 프롬프트 종류: ai_challenge, 
  점수: 84,
  소요시간: 76.041초

#####
6 프롬프트 종류: emotional_appeal, 
  점수: 89,
  소요시간: 129.347초

#####
7 프롬프트 종류: costar, 
  점수: 18,
  소요시간: 533.599초



### [LOG] 2.2 Result

In [84]:
from pathlib import Path
import pandas as pd
data_path = Path('data/week4_advanced')
csv_list = [path for path in data_path.rglob("*.csv")]
pred_dict = {}
for name in list(total_prompt_dict.keys()):
    for csv in csv_list:
        if name in str(csv):
            break
    pred_dict[name] = pd.read_csv(csv)
    pred_dict[name] = pred_dict[name].drop(columns=['Unnamed: 0'])

In [85]:
# 각 프롬프트에 대한 결과를 출력해놓았습니다.

from IPython.display import display
for i, (name, df) in enumerate(pred_dict.items()):
    print(f"{i+1}번째 프롬프트 {name} 의 결과")
    display(df)
    print("\n"*5)
    

1번째 프롬프트 minimum_prompt 의 결과


,pred,answer,score,input_tokens,output_tokens,response,time,result
0,4,4,2,827,6,정답: 4번,39.181911,True
1,5,5,3,1145,6,정답: 5번,39.181911,True
2,1,1,2,841,6,정답: 1번,39.181911,True
3,4,4,2,1684,6,정답: 4번,39.181911,True
4,5,5,2,1651,6,정답: 5번,39.181911,True
5,3,3,2,1747,6,정답: 3번,39.181911,True
6,2,2,2,1696,6,정답: 2번,39.181911,True
7,5,5,3,1950,6,정답: 5번,39.181911,True
8,2,2,2,1575,6,정답: 2번,39.181911,True
9,4,4,2,1073,6,정답: 4번,39.181911,True








2번째 프롬프트 zero_shot_cot 의 결과


,pred,answer,score,input_tokens,output_tokens,response,time,result
0,4,4,2,840,99,"정답: 4번\n\n제시된 지문에서는 ""독자의 배경지식이나 관점 등의 독자 요인, 읽...",99.696625,True
1,5,5,3,1158,6,정답: 5번,99.696625,True
2,1,1,2,854,6,정답: 1번,99.696625,True
3,4,4,2,1697,6,정답: 4번,99.696625,True
4,5,5,2,1664,6,정답: 5번,99.696625,True
5,3,3,2,1760,315,정답을 찾기 위해 각 선택지를 지문과 비교하여 검토해보겠습니다.\n\n1번 문장은 ...,99.696625,True
6,2,2,2,1709,83,정답: 2번\n\n이수광은 주자학뿐만 아니라 다른 학문에 대해서도 열린 태도를 가지...,99.696625,True
7,1,5,3,1963,233,정답: 5번\n\n해설:\n- 1번: 임원경제지는 현실의 문제를 개선하려는 목적의식...,99.696625,False
8,2,2,2,1588,411,질문에서 요구하는 바는 문맥상 적절하지 않은 단어를 찾는 것입니다.\n\n1. ⓐ(...,99.696625,True
9,1,4,2,1086,220,"정답: 4번\n\n- 1번의 경우, 법령의 요건과 효과에 불확정 개념이 사용될 수 ...",99.696625,False








3번째 프롬프트 plan_and_solve 의 결과


...

,pred,answer,score,input_tokens,output_tokens,response,time,result
0,4,4,2,832,82,"윗글에서는 독자의 배경지식, 관점, 읽기 환경, 과제가 독자의 의미 구성에 영향을 ...",76.041067,True
1,5,5,3,1150,6,정답: 5번,76.041067,True
2,1,1,2,846,66,"정답: 1번\n\n'㉠ 다른 독자와 소통하는 즐거움'을 언급하였으므로, 다른 독자와...",76.041067,True
3,4,4,2,1689,6,정답: 4번,76.041067,True
4,5,5,2,1656,6,정답: 5번,76.041067,True
5,3,3,2,1752,6,정답: 3번,76.041067,True
6,2,2,2,1701,97,정답: 2번\n\n이수광은 지봉유설을 통해 주자학뿐 아니라 다양한 지식을 배우고 확...,76.041067,True
7,5,5,3,1955,6,정답: 5번,76.041067,True
8,2,2,2,1580,312,"지문에서 제공하는 정보에 따르면, 각 괄호 ⓐ~ⓔ에 넣을 수 있는 적절한 단어들이 ...",76.041067,True
9,4,4,2,1078,99,"정답: 4번\n\n지문에 따르면, 불확정 개념이 사용된 행정 법령에 근거한 행정 작...",76.041067,True








6번째 프롬프트 emotional_appeal 의 결과


,pred,answer,score,input_tokens,output_tokens,response,time,result
0,4,4,2,816,4,정답: None,129.346741,True
1,5,5,3,1134,6,정답: 5번,129.346741,True
2,1,1,2,830,106,지문의 ㉠ 부분에서는 '다른 독자와 소통하는 즐거움'을 언급하고 있습니다. 이와 관...,129.346741,True
3,4,4,2,1673,6,정답: 4번,129.346741,True
4,5,5,2,1640,78,정답: 5번\n\n5번 선택지는 지문에서 언급된 중국의 유서 편찬 방식과 다릅니다....,129.346741,True
5,3,3,2,1736,6,정답: 3번,129.346741,True
6,2,2,2,1685,6,정답: 2번,129.346741,True
7,5,5,3,1939,6,정답: 5번,129.346741,True
8,2,2,2,1564,266,문맥상 적절하지 않은 단어를 찾기 위해 각 선택지를 확인해 보겠습니다.\n\n1번 ...,129.346741,True
9,4,4,2,1062,55,정답: 4번\n\n지문에서는 불확정 개념이 사용된 법령에 근거한 행정 작용은 대개 ...,129.346741,True








7번째 프롬프트 costar 의 결과


,pred,answer,score,input_tokens,output_tokens,response,time,result
0,1,4,2,1342,478,1. **지문 요약** \n - 사람들은 독서를 통해 필자와 소통하며 즐거움을...,533.598737,False
1,1,5,3,1660,550,"1. **지문 요약**\n - 독자는 책과 소통하며 즐거움을 느끼고, 이는 필자...",533.598737,False
2,1,1,2,1356,462,1. **지문 요약**\n - 지문에서는 사람들이 책을 읽는 이유 중 하나로 소...,533.598737,True
3,1,4,2,2199,610,1. **지문 요약**\n - (가)에서는 중국과 조선의 전통 유서 편찬 방식과...,533.598737,False
4,1,5,2,2166,681,1. **지문 요약** \n- 유서는 고금의 서적에서 자료를 수집하고 항목별로 분...,533.598737,False
5,1,3,2,2262,698,1. **지문 요약**\n\n - (가): 유서(類書)는 다양한 정보를 수집하고...,533.598737,False
6,1,2,2,2211,613,1. **지문 요약**\n - (가)에서는 중국에서 유래된 유서가 조선에 어떻게...,533.598737,False
7,1,5,3,2465,562,1. **지문 요약**\n - (가)에서는 조선 시대에 유서가 어떤 방식으로 발...,533.598737,False
8,1,2,2,2090,701,1. **지문 요약** \n 지문은 중국의 유서 전통과 조선시대 유서 편찬의 ...,533.598737,False
9,1,4,2,1588,720,1. **지문 요약**\n - 지문에서는 법령의 조문이 요건과 효과로 구성된 조...,533.598737,False


### [My Code] 2.2 Result

- 각 프롬프트 별로 정답일 때와 오답일 때의 예시입니다.

In [ ]:
# 각 프롬프트 타입에 대한 정답과 오답의 예시
for i, name in enumerate(pred_dict.keys()):
    true_response = pred_dict[name][pred_dict[name]['result']].iloc[0].response
    true_answer = pred_dict[name][pred_dict[name]['result']].iloc[0].answer
    true_index = pred_dict[name][pred_dict[name]['result']].iloc[0].name

    false_response = pred_dict[name][~pred_dict[name]['result']].iloc[0].response
    false_answer = pred_dict[name][~pred_dict[name]['result']].iloc[0].answer
    false_index = pred_dict[name][~pred_dict[name]['result']].iloc[0].name
    print("#####")
    print(f"{i+1}번째: {name} 프롬프트의 정답 예시", end = "\n\n")
    print("  프롬프트")
    print(total_prompt_dict[name][true_index])
    print("  예측")
    print(true_response)
    print("  정답")
    print(true_answer, end = "\n\n")
    print(f"{i+1}번째: {name} 프롬프트의 오답 예시")
    print("  프롬프트")
    print(total_prompt_dict[name][false_index])
    print("  예측")
    print(false_response)
    print("  정답")
    print(false_answer, end = "\n\n")

#####
1번째: minimum_prompt 프롬프트의 정답 예시

  프롬프트

    국어 시험 문제를 푸는 똑똑한 학생으로서 다음 문제의 답을 구하세요.
    지문을 읽고, 질문에 대한 답을 1부터 5까지의 선택지 중에 한 개만 골라서 대답해야 합니다.
    최종 정답은 반드시 "정답: <<번호>>번" 형태로 작성해주세요.

    정답 작성 예시:
    정답: 3번

    지문: 사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다.독자는 독서를 통해 책과 소통하는 즐거움을 경험한다. 독서는필자와 간접적으로 대화하는 소통 행위이다. 독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다. 이때 같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다.[A] (이러한 소통은 독자가 책의 내용에 대해 질문하고 답을 찾아내는 과정에서 가능해진다. 독자는 책에서 답을 찾는 질문, 독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다. 전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고, 책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로 구성할 수도 있다. 또한 후자의 경우 책에는 없는 독자의 경험에서 답을 찾을 수 있다. 이런 질문들을 풍부히 생성하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다.)한편 독자는 ㉠ (다른 독자와 소통하는 즐거움을 경험할 수도 있다.) 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당한다. 비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차

## [My Code] 3 Prediction - RAG

### [My Code] 3.1 Paragraph Retrieval

In [ ]:
import openai
import json
import os
from dotenv import load_dotenv
import prompts

from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

def retrieve_relevant_paragraphs(question, choices, question_plus=None):
    query = question
    if question_plus:
        query += " " + question_plus
    query += " " + " ".join(choices)
    
    results = retriever.get_relevant_documents(query)
    return results

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,       
    chunk_overlap=100,    
    separators=[".", "\n", " ", ""]  
)


embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

total_ragged_dict = {}
for name, value in prompts.prompts.items():
    prompt_lst = []
    for data in total_result:

        paragraph = data['paragraph']
        split_texts = text_splitter.split_text(paragraph)
        documents = [Document(page_content=chunk, metadata={}) for chunk in split_texts]

        vector_store = FAISS.from_documents(documents, embedding_model)
        retriever = vector_store.as_retriever(search_type="similarity", search_kwargs={"k": 10})
        retrieved_docs = retrieve_relevant_paragraphs(question, choices, question_plus)

        paragraph = ''
        for i, doc in enumerate(retrieved_docs):
            paragraph = paragraph + f"{i+1}번째 문서: {doc.page_content}"

        prompt = value.format(
            paragraph=paragraph,
            question=data['question'],
            question_plus=data['question_plus'],
            choices1=data['choices'][0],
            choices2=data['choices'][1],
            choices3=data['choices'][2],
            choices4=data['choices'][3],
            choices5=data['choices'][4],
        )
        prompt_lst.append(prompt)
    total_ragged_dict[name] = prompt_lst

### [My Code] 3.2 Prediction-RAG

In [77]:
# OpenAI API 호출
from openai import OpenAI
import random
import tiktoken
import time
import pandas as pd

tokenizer = tiktoken.encoding_for_model("gpt-4o")
client = OpenAI()

def prediction(prompt, model="gpt-4o"):
    completion = client.chat.completions.create(
        model=model,
        messages=[{"role": "user","content": prompt}]
    )
    return completion.choices[0].message

def normalize_answer(text):
    result = text.content
    result = result.split("정답:")[-1]
    for num in range(1, 6):
        if str(num) in result:
            return num
        
    return random.randint(1,5)

pred_dict = {}
for i, (name, prompt_lst) in enumerate(total_ragged_dict.items()):
    start_time = time.time()
    pred_lst = []
    answer_lst = []
    score_lst = []
    input_token_lst = []
    output_token_lst = []
    response_lst = []
    result_lst = []
    for prompt, problem  in zip(prompt_lst, total_result):
        result = prediction(prompt)
        pred = normalize_answer(result)

        pred_lst.append(pred)
        answer_lst.append(problem['answer'])
        score_lst.append(problem['score'])
        input_token_lst.append(len(tokenizer.encode(prompt)))
        output_token_lst.append(len(tokenizer.encode(result.content)))
        response_lst.append(result.content)

    end_time = time.time()
    execution_time = end_time - start_time
    pred_dict[name] = pd.DataFrame({
        'pred':pred_lst, 
        'answer':answer_lst, 
        'score':score_lst,
        'input_tokens':input_token_lst,
        'output_tokens': output_token_lst,
        'response' : response_lst,
        'time' : execution_time,
        })
    pred_dict[name]['result'] = pred_dict[name]['pred'].apply(lambda x: int(x)) == pred_dict[name]['answer'].apply(lambda x: int(x))
    pred_dict[name].to_csv(f"data/week4_ragged_advanced/{name}.csv")
    total_score = (pred_dict[name]['result'] * pred_dict[name]['score']).sum()
    print("#####")
    print(f"{i+1} 프롬프트 종류: {name}, \n  점수: {total_score},\n  소요시간: {execution_time:.3f}초", end="\n\n")

#####
1 프롬프트 종류: minimum_prompt, 
  점수: 72,
  소요시간: 46.588초

#####
2 프롬프트 종류: zero_shot_cot, 
  점수: 61,
  소요시간: 182.521초

#####
3 프롬프트 종류: plan_and_solve, 
  점수: 68,
  소요시간: 368.473초

#####
4 프롬프트 종류: format_specified, 
  점수: 51,
  소요시간: 514.915초

#####
5 프롬프트 종류: ai_challenge, 
  점수: 72,
  소요시간: 133.906초

#####
6 프롬프트 종류: emotional_appeal, 
  점수: 77,
  소요시간: 136.138초

#####
7 프롬프트 종류: costar, 
  점수: 20,
  소요시간: 452.896초



### [LOG] 3.3 Result

In [86]:
from pathlib import Path
import pandas as pd
data_path = Path('data/week4_ragged_advanced')
csv_list = [path for path in data_path.rglob("*.csv")]
pred_dict = {}
for name in list(total_ragged_dict.keys()):
    for csv in csv_list:
        if name in str(csv):
            break
    pred_dict[name] = pd.read_csv(csv)
    pred_dict[name] = pred_dict[name].drop(columns=['Unnamed: 0'])

In [87]:
from IPython.display import display
for i, (name, df) in enumerate(pred_dict.items()):
    print(f"{i+1}번째 프롬프트 {name} 의 결과")
    display(df)
    print("\n"*5)
    

1번째 프롬프트 minimum_prompt 의 결과


,pred,answer,score,input_tokens,output_tokens,response,time,result
0,4,4,2,1163,6,정답: 4번,46.587786,True
1,5,5,3,1481,6,정답: 5번,46.587786,True
2,1,1,2,1177,6,정답: 1번,46.587786,True
3,4,4,2,1416,6,정답: 4번,46.587786,True
4,5,5,2,1383,6,정답: 5번,46.587786,True
5,3,3,2,1479,6,정답: 3번,46.587786,True
6,2,2,2,1428,6,정답: 2번,46.587786,True
7,5,5,3,1682,6,정답: 5번,46.587786,True
8,2,2,2,1307,6,정답: 2번,46.587786,True
9,4,4,2,1425,6,정답: 4번,46.587786,True








2번째 프롬프트 zero_shot_cot 의 결과


,pred,answer,score,input_tokens,output_tokens,response,time,result
0,5,4,2,1176,258,"질문에 답하기 위해서는 주어진 지문에서 일관된 진술이나 의미가 있는지, 선택지의 각...",182.520674,False
1,5,5,3,1494,6,정답: 5번,182.520674,True
2,1,1,2,1190,177,"주어진 와 ㉠은 ""다른 독자와 소통하는 즐거움을 경험할 수도 있다""라는 내용에 해당...",182.520674,True
3,4,4,2,1429,6,정답: 4번,182.520674,True
4,5,5,2,1396,6,정답: 5번,182.520674,True
5,3,3,2,1492,6,정답: 3번,182.520674,True
6,2,2,2,1441,282,"지문과 질문을 바탕으로 분석해보면, '이수광'은 주자학뿐만 아니라 서학을 포함한 다...",182.520674,True
7,3,5,3,1695,461,지문과 문제를 분석하여 선택지를 검토해 보겠습니다.\n\n1. **지문에서의 내용 ...,182.520674,False
8,2,2,2,1320,246,문맥을 통해 지문에서 각 단어가 사용된 방식과 그에 맞는 제시된 단어를 비교해 보겠...,182.520674,True
9,4,4,2,1438,261,지문을 분석해 보면:\n\n1. 법령의 요건과 효과에 모두 불확정 개념이 사용될 수...,182.520674,True








3번째 프롬프트 plan_and_solve 의 결과


...

,pred,answer,score,input_tokens,output_tokens,response,time,result
0,1,4,2,1168,338,지문에서 제공된 정보들을 통해 각 선택지를 검토해보겠습니다.\n\n1번 - 지문에서...,133.906028,False
1,5,5,3,1486,6,정답: 5번,133.906028,True
2,1,1,2,1182,219,"지문과 선택지를 분석해 본 결과, ㉠에 해당하는 ""다른 독자와 소통하는 즐거움""은 ...",133.906028,True
3,4,4,2,1421,6,정답: 4번,133.906028,True
4,5,5,2,1388,6,정답: 5번,133.906028,True
5,3,3,2,1484,504,"정답을 도출하기 위해 지문과 선택지를 분석해보겠습니다. \n\n먼저, 주어진 지문에...",133.906028,True
6,2,2,2,1433,6,정답: 2번,133.906028,True
7,5,5,3,1687,204,정답: 5번\n\n위 지문에서는 조선 후기 실학자들과 그들의 유서에 대한 내용을 드...,133.906028,True
8,2,2,2,1312,138,지문에서 제공된 문맥상 각 단어의 의미를 평가해보면 아래와 같은 바꿔 쓸 수 있는 ...,133.906028,True
9,4,4,2,1430,116,정답: 4번\n\n4번: 불확정 개념이 사용된 행정 법령에 근거한 행정 작용은 재량...,133.906028,True








6번째 프롬프트 emotional_appeal 의 결과


,pred,answer,score,input_tokens,output_tokens,response,time,result
0,4,4,2,1152,6,정답: 4번,136.138321,True
1,5,5,3,1470,6,정답: 5번,136.138321,True
2,1,1,2,1166,88,윗글에서는 ㉠이 '다른 독자와 소통하는 즐거움'이라고 설명합니다. 이를 바탕으로 각...,136.138321,True
3,4,4,2,1405,124,"지문과 선택지를 분석해 보면, (가)는 유서의 기원과 개념, 사용목적에 대해 설명하...",136.138321,True
4,5,5,2,1372,224,질문에서 [A]에 대한 이해로 적절하지 않은 것을 찾으라고 했습니다. [A]에서는 ...,136.138321,True
5,3,3,2,1468,6,정답: 3번,136.138321,True
6,2,2,2,1417,88,"지문에서 이수광은 ""주자학뿐 아니라 다른 학문에 대해서도 열린 태도를 가지고 있다""...",136.138321,True
7,5,5,3,1671,386,"문제를 풀기 위해 (가)와 (나) 부분의 서술을 살펴보면, (가)에서는 중국과 조선...",136.138321,True
8,2,2,2,1296,278,"문맥에서 보았을 때, 각 단어가 대치될 수 있는지 확인해보면 다음과 같습니다.\n\...",136.138321,True
9,4,4,2,1414,216,"주어진 지문들과 선택지를 살펴보면, 불확정 개념에 관한 설명들이 나열되어 있습니다....",136.138321,True








7번째 프롬프트 costar 의 결과


,pred,answer,score,input_tokens,output_tokens,response,time,result
0,1,4,2,1678,826,1. **지문 요약**\n - 지문은 독서의 소통적 측면에 대해 논의하고 있으며...,452.89591,False
1,1,5,3,1996,655,1. **지문 요약**\n - 지문은 독서의 과정에서 독자가 책과 소통하여 의미...,452.89591,False
2,1,1,2,1692,532,"1. **지문 요약**\n - 지문은 독서의 즐거움에 대해 설명하며, 독자가 책...",452.89591,True
3,1,4,2,1931,553,1. **지문 요약**\n - (가) 중국에서 시작된 유서는 과거의 서적에서 자...,452.89591,False
4,5,5,2,1898,510,1. **지문 요약**\n\n - 유서는 일반적으로 책에서 필요한 부분을 발췌...,452.89591,True
5,1,3,2,1994,457,1. **지문 요약**\n - 조선 후기 실학자들은 주자학 외의 다양한 지식을 ...,452.89591,False
6,1,2,2,1943,553,1. **지문 요약**\n - 조선 후기 실학자들은 주자학의 관념적 사유에 국한...,452.89591,False
7,1,5,3,2197,743,1. **지문 요약**\n - 주어진 지문에서는 조선 후기 실학자들이 편찬한 유...,452.89591,False
8,1,2,2,1822,684,### 1. **지문 요약**\n\n- **실학자의 유서:** 조선 후기 실학자들은...,452.89591,False
9,1,4,2,1940,582,1. **지문 요약**\n - 법조문에는 불확정 개념이 사용되어 상황에 따라 해...,452.89591,False


In [88]:
# 각 프롬프트 타입에 대한 정답과 오답의 예시
for i, name in enumerate(pred_dict.keys()):
    true_response = pred_dict[name][pred_dict[name]['result']].iloc[0].response
    true_answer = pred_dict[name][pred_dict[name]['result']].iloc[0].answer
    true_index = pred_dict[name][pred_dict[name]['result']].iloc[0].name

    false_response = pred_dict[name][~pred_dict[name]['result']].iloc[0].response
    false_answer = pred_dict[name][~pred_dict[name]['result']].iloc[0].answer
    false_index = pred_dict[name][~pred_dict[name]['result']].iloc[0].name
    print("#####")
    print(f"{i+1}번째: {name} 프롬프트의 정답 예시", end = "\n\n")
    print("  프롬프트")
    print(total_prompt_dict[name][true_index])
    print("  예측")
    print(true_response)
    print("  정답")
    print(true_answer, end = "\n\n")
    print(f"{i+1}번째: {name} 프롬프트의 오답 예시")
    print("  프롬프트")
    print(total_prompt_dict[name][false_index])
    print("  예측")
    print(false_response)
    print("  정답")
    print(false_answer, end = "\n\n")

#####
1번째: minimum_prompt 프롬프트의 정답 예시

  프롬프트

    국어 시험 문제를 푸는 똑똑한 학생으로서 다음 문제의 답을 구하세요.
    지문을 읽고, 질문에 대한 답을 1부터 5까지의 선택지 중에 한 개만 골라서 대답해야 합니다.
    최종 정답은 반드시 "정답: <<번호>>번" 형태로 작성해주세요.

    정답 작성 예시:
    정답: 3번

    지문: 사람들이 지속적으로 책을 읽는 이유 중 하나는 즐거움이다. 독서의 즐거움에는 여러 가지가 있겠지만 그 중심에는 ‘소통의 즐거움’이 있다.독자는 독서를 통해 책과 소통하는 즐거움을 경험한다. 독서는필자와 간접적으로 대화하는 소통 행위이다. 독자는 자신이 속한사회나 시대의 영향 아래 필자가 속해 있거나 드러내고자 하는 사회나 시대를 경험한다. 직접 경험하지 못했던 다양한 삶을 필자를 매개로 만나고 이해하면서 독자는 더 넓은 시야로 세계를바라볼 수 있다. 이때 같은 책을 읽은 독자라도 독자의 배경지식이나 관점 등의 독자 요인, 읽기 환경이나 과제 등의 상황 요인이 다르므로, 필자가 보여 주는 세계를 그대로 수용하지 않고 저마다 소통 과정에서 다른 의미를 구성할 수 있다.[A] (이러한 소통은 독자가 책의 내용에 대해 질문하고 답을 찾아내는 과정에서 가능해진다. 독자는 책에서 답을 찾는 질문, 독자 자신에게서 답을 찾는 질문 등을 제기할 수 있다. 전자의 경우 책에 명시된 내용에서 답을 발견할 수 있고, 책의 내용들을 관계 지으며 답에 해당하는 내용을 스스로 구성할 수도 있다. 또한 후자의 경우 책에는 없는 독자의 경험에서 답을 찾을 수 있다. 이런 질문들을 풍부히 생성하고 주체적으로 답을 찾을 때 소통의 즐거움은 더 커진다.)한편 독자는 ㉠ (다른 독자와 소통하는 즐거움을 경험할 수도 있다.) 책과의 소통을 통해 개인적으로 형성한 의미를 독서 모임이나 독서 동아리 등에서 다른 독자들과 나누는 일이 이에 해당한다. 비슷한 해석에 서로 공감하며 기존 인식을 강화하거나 관점의 차

## 4 [My Code] Conclusion

- gpt 4o 기준 Emotional appeal prompt와 minimum prompt, ai challenge prompt가 가장 성능이 좋았습니다.

- 가장 상세하게 prompt를 제공한 costar 기법은 전혀 효과가 없었습니다.

- 문서를 일부 검색해서 보내었을 땐 그렇지 않고 전체 문제를 주었을 때보다 성능이 떨어졌습니다.

- 문서 검색 기법이 효과가 없었던 것은 gpt-4o가 128000개의 토큰을 요구하는 데에 반해, 전체 프롬프트를 다 주어도 3천개 토큰을 절대 넘지 않기 때문에, gpt에겐 전체 paragraph가 오히려 매우 적합한 양이었다고 해석했습니다.

- 따라서 유사도 기반 검색이 오히려 적합한 양의 prompt를 줄임으로써 문제를 키웠다고 해석하였습니다.